# Creating arrays along the coast

In [11]:
from xmitgcm import open_mdsdataset
from MITgcmutils import rdmds
from MITgcmutils import mds
from MITgcmutils import diagnostics
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dat
from matplotlib import cm
import cmocean
import xarray as xr
from math import radians, cos, sin, asin, sqrt
import netCDF4 as nc
import warnings
warnings.filterwarnings("ignore")

In [12]:
data_dirWITH= '/Volumes/Trillian/SVB/exp06_512x612x100_ORL_SVB/01_SVB_febTS/'
data_dirNO= '/Volumes/Trillian/SVB/exp06_512x612x100_ORL/01_noSVB_febTS/'

In [ ]:
def unstagger(ugrid, vgrid):
    """ Interpolate u and v component values to values at grid cell centres (from D.Latornell for NEMO output).
    The shapes of the returned arrays are 1 less than those of
    the input arrays in the y and x dimensions.
    :arg ugrid: u velocity component values with axes (..., y, x)
    :type ugrid: :py:class:`numpy.ndarray`
    :arg vgrid: v velocity component values with axes (..., y, x)
    :type vgrid: :py:class:`numpy.ndarray`
    :returns u, v: u and v component values at grid cell centres
    :rtype: 2-tuple of :py:class:`numpy.ndarray`
    """
    u = np.add(ugrid[..., :-1], ugrid[..., 1:]) / 2
    v = np.add(vgrid[..., :-1, :], vgrid[..., 1:, :]) / 2
    return u, v

In [13]:
def findlonlat(hFacCuse,d):

    nx = 512
    ny = 612
    ind30=170+50 #30° N
    indlon = 50 # Lon of land at N boundary

    lon_inds_off = np.argmax(np.squeeze(hFacCuse[d,:,::-1]), axis=1)


    lon_inds = np.ones_like(lon_inds_off[ind30:])*nx - lon_inds_off[ind30:]
    lat_inds = np.ones_like(lon_inds)*ind30 + np.arange(len(lon_inds))

    lat_inds_off = np.argmax(np.squeeze(hFacCuse[d,::-1,:]), axis=0)



    lat_inds_2 = np.ones_like(lat_inds_off[indlon:])*ny - lat_inds_off[indlon:]
    lon_inds_2 = np.ones_like(lat_inds)*indlon + np.arange(len(lat_inds))

    p=0

    k=np.ones_like(lon_inds)
    l=np.ones_like(lat_inds)

    for i in np.arange(len(lon_inds_2)):
        if lon_inds_2[i] in lon_inds:
            o=0
        else:
            p=p+1
            k[p]=lon_inds_2[i]
            l[p]=lat_inds_2[i]
        
    k=k[k!=1]
    l=l[l!=1]

    lat_fix=np.append(lat_inds,l)
    lon_fix=np.append(lon_inds,k)
    
    lon_fix=lon_fix[lat_fix >=220]
    lat_fix=lat_fix[lat_fix >=220]
    
    lon_fix=lon_fix[lat_fix <=480]
    lat_fix=lat_fix[lat_fix <=480]

    print("Done")
    
    return(lat_fix, lon_fix)

In [14]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [16]:
def loadingcoastpts(lon_fix,lat_fix,Z,prevalue,name,name2,name3,nt,t,FILENAME):
    
    dist_array = np.zeros(len(lon_fix))
    p=0
    for jj,ii in zip(lon_fix, lat_fix):
        lat1 = LAT[ii-2]
        lon1 = LON[jj-2]
        lat2 = LAT[ii-1]
        lon2 = LON[jj-1]
        p=p+1
        dist_array[p-1]=  haversine(lat1, lon1, lat2, lon2)
    
    dist_cummul = np.cumsum(dist_array)
    
    value=prevalue.values
    val = np.zeros((nt,len(lon_fix)))
    
    p=0
    
    for ii,jj in zip(lon_fix, lat_fix):
        p=p+1
        val[:,p-1] = value[:,jj-1,ii-1]
     
    ds = xr.Dataset({name: (("time","x"), val),
                    name2:(("x"), lon_fix),
                    name3:(("x"), lat_fix)
                    },
                    coords ={
                    "x" : dist_cummul,
                    "time": t,
                    },
                   )
    
    ds.to_netcdf(FILENAME)
    
def varsalongcoasts(hFacCuse,LAT,LON,Z,valvar,name,name2,name3,nt,t,FILENAME):
    Zdyn=Z[:71].values
    val = np.zeros((nt,71,354))
    
    for dep in np.arange(0,71,1):
        lat_fix1, lon_fix1=findlonlat(hFacCuse,dep)

        lonlat2=np.zeros((len(lon_fix1),2),dtype=int)
        lonlat2[:,0]=lon_fix1
        lonlat2[:,1]=lat_fix1

        lonlat=lonlat2[lonlat2[:, 1].argsort()]

        lat_fix=lonlat[:354, 1]
        lon_fix=lonlat[:354, 0]

        dist_array = np.zeros(len(lon_fix))
        p=0
        for jj,ii in zip(lon_fix, lat_fix):
            lat1 = LAT[ii-2]
            lon1 = LON[jj-2]
            lat2 = LAT[ii-1]
            lon2 = LON[jj-1]
            p=p+1
            dist_array[p-1]=  haversine(lat1, lon1, lat2, lon2)

            dist_cummul = np.cumsum(dist_array)

        for h in range(nt):
            p=0
            for ii,jj in zip(lon_fix, lat_fix):
                p=p+1
                value=valvar[h-1,dep,jj-1,ii-1]
                val[h-1,dep,p-1] = value
         
    ds = xr.Dataset({name: (("time","z","x"), val),                 
                     name2:(("x"), lon_fix),
                     name3:(("x"), lat_fix)
                    },
                coords ={
                    "x" : dist_cummul,
                    "z" : Zdyn,
                    "time": t,
                },
                )
    ds.to_netcdf(FILENAME)
        


    

In [17]:
def loadNetCDFs(varname):
    dsw=[]
    dsn=[]
    for i in np.arange(1,8,1):
        
        pathn='/Volumes/Trillian/SVB/exp06_512x612x100_ORL/01_noSVB_febTS/'+ str(varname)+'noSVB'+ str(2+i)+'_'+ str(3+i) +'.nc'
        pathw='/Volumes/Trillian/SVB/exp06_512x612x100_ORL_SVB/01_SVB_febTS/'+ str(varname)+'withSVB'+ str(2+i)+'_'+ str(3+i) +'.nc'
        
        dswin  = xr.open_dataset(pathw)
        dsnin = xr.open_dataset(pathn)
        
        dsw.append(dswin)
        dsn.append(dsnin)
        
    return dsw, dsn

def loadvar(dsw,dsn):
    
    varw=[]
    varn=[]
    for hej in np.arange(0,8,1):
        
        variable=dsw[hej].WVEL.values
        variablen=dsn[hej].WVEL.values
        varw.append(variable)
        varn.append(variablen)
        print('plopp')
    return varw, varn
    

In [18]:
dsw,dsn=loadNetCDFs('DYNVARS')

In [19]:
LAT=dsw[0].YC
LON=dsw[0].XC-360
Zdyn = dsw[0].Z
hFacCw = dsw[0].hFacC
hFacCn = dsn[0].hFacC
hFacCusew=hFacCw.values
hFacCusen=hFacCn.values

varname='VVEL'

l=7
pathn='/Volumes/Trillian/SVB/exp06_512x612x100_ORL/01_noSVB_febTS/'+ str(varname)+'ACnoSVB'+ str(2+l)+'_'+ str(3+l) +'.nc'
pathw='/Volumes/Trillian/SVB/exp06_512x612x100_ORL_SVB/01_SVB_febTS/'+ str(varname)+'ACwithSVB'+ str(2+l)+'_'+ str(3+l) +'.nc'
i=6
Vw=dsw[6].VVEL.values
Vn=dsn[6].VVEL.values

TIMEdyn=dsw[6].time
ntdyn = np.size(TIMEdyn)

varsalongcoasts(hFacCusew,LAT,LON,Zdyn,Vw,'Vw','lonAC','latAC',ntdyn,TIMEdyn,pathw)
varsalongcoasts(hFacCusen,LAT,LON,Zdyn,Vn,'Vn','lonAC','latAC',ntdyn,TIMEdyn,pathn)



KeyboardInterrupt: 

In [10]:
LAT=dsw[0].YC
LON=dsw[0].XC-360
Zdyn = dsw[0].Z
hFacCw = dsw[0].hFacC
hFacCn = dsn[0].hFacC
hFacCusew=hFacCw.values
hFacCusen=hFacCn.values

varname='UVEL'

for l in np.arange(0,7,1):

    pathn='/media/amelia/Trillian/SVB/exp06_512x612x100_ORL/01_noSVB_febTS/'+ str(varname)+'ACnoSVB'+ str(2+l)+'_'+ str(3+l) +'.nc'
    pathw='/media/amelia/Trillian/SVB/exp06_512x612x100_ORL_SVB/01_SVB_febTS/'+ str(varname)+'ACwithSVB'+ str(2+l)+'_'+ str(3+l) +'.nc'

    Uw=dsw[l].UVEL.values
    Un=dsn[l].UVEL.values

    TIMEdyn=dsw[l].time
    ntdyn = np.size(TIMEdyn)

    varsalongcoasts(hFacCusew,LAT,LON,Zdyn,Uw,'Uw','lonAC','latAC',ntdyn,TIMEdyn,pathw)
    varsalongcoasts(hFacCusen,LAT,LON,Zdyn,Un,'Un','lonAC','latAC',ntdyn,TIMEdyn,pathn)


KeyboardInterrupt: 

In [ ]:
varsalongcoasts(hFacCuse,LAT,LON,Zdyn,ww,'ww','lonAC','latAC',ntdyw,TIMEdyw,'WvelwithSVB.nc')
varsalongcoasts(hFacCuse,LAT,LON,Zdyn,uw,'uw','lonAC','latAC',ntdyw,TIMEdyw,'/Volumes/Trillian/SVB/exp06_512x612x100_ORL/01_noSVB_febTS/UvelwithSVB.nc')
varsalongcoasts(hFacCuse,LAT,LON,Zdyn,vw,'vw','lonAC','latAC',ntdyw,TIMEdyw,'/Volumes/Trillian/SVB/exp06_512x612x100_ORL/01_noSVB_febTS/VvelwithSVB.nc')
varsalongcoasts(hFacCuse,LAT,LON,Zdyn,Sw,'Sw','lonAC','latAC',ntdyw,TIMEdyw,'/Volumes/Trillian/SVB/exp06_512x612x100_ORL/01_noSVB_febTS/SaltwithSVB.nc')
varsalongcoasts(hFacCuse,LAT,LON,Zdyn,Tw,'Tw','lonAC','latAC',ntdyw,TIMEdyw,'/Volumes/Trillian/SVB/exp06_512x612x100_ORL/01_noSVB_febTS/ThetawithSVB.nc')

varsalongcoasts(hFacCuse,LAT,LON,Zdyn,wn,'wn','lonAC','latAC',ntdyn,TIMEdyn,'/Volumes/Trillian/SVB/exp06_512x612x100_ORL/01_noSVB_febTS/WvelnoSVB.nc')
varsalongcoasts(hFacCuse,LAT,LON,Zdyn,un,'un','lonAC','latAC',ntdyn,TIMEdyn,'/Volumes/Trillian/SVB/exp06_512x612x100_ORL/01_noSVB_febTS/UvelnoSVB.nc')
varsalongcoasts(hFacCuse,LAT,LON,Zdyn,vn,'vn','lonAC','latAC',ntdyn,TIMEdyn,'/Volumes/Trillian/SVB/exp06_512x612x100_ORL/01_noSVB_febTS/VvelnoSVB.nc')
varsalongcoasts(hFacCuse,LAT,LON,Zdyn,Sn,'Sn','lonAC','latAC',ntdyn,TIMEdyn,'/Volumes/Trillian/SVB/exp06_512x612x100_ORL/01_noSVB_febTS/SaltnoSVB.nc')
varsalongcoasts(hFacCuse,LAT,LON,Zdyn,Tn,'Tn','lonAC','latAC',ntdyn,TIMEdyn,'/Volumes/Trillian/SVB/exp06_512x612x100_ORL/01_noSVB_febTS/ThetanoSVB.nc')

In [ ]:
lonlat2=np.zeros((len(lon_fix),2),dtype=int)
lonlat2[:,0]=lon_fix
lonlat2[:,1]=lat_fix
lonlat=lonlat2[lonlat2[:, 1].argsort()]
lat_fix=lonlat[:, 1]
lon_fix=lonlat[:, 0]

In [ ]:
for ii,jj in zip(lon_fix,lat_fix):
    if depth[jj-1,ii-1] == 0:
        lon_fix[np.where(lon_fix == ii)]=ii-1
        lat_fix[np.where(lat_fix == jj)]=jj-1
        print(str(depth[jj-2,ii-2].values))
        

In [ ]:
lat_fix, lon_fix=findlonlat(hFacCuse,30)

In [ ]:

d=55
lat_inds_off = np.argmax(np.squeeze(hFacCuse[d,::-1,:]), axis=0)
lon_inds_off = np.argmax(np.squeeze(hFacCuse[d,:,::-1]), axis=1)

In [ ]:
depth = dsw.Depth
LAT = dsw.YC
LON = dsw.XC-360

hFacCw = dsw.hFacC
hFacCn = dsn.hFacC


maskcw=dsw.maskInC==False
maskcn=dsn.maskInC==False


ind30=170+50 #30° N
ind307=170+100 #30.7° N
ind314=170+150 #31.4° N
ind32=170+200 #32° N

indlon = 50 # Lon of land at N boundary

nx = 512
ny = 612
nz = 100

hFacCuse=hFacCw.values

In [ ]:
loadingcoastpts(lon_fix,lat_fix,Zeta,etaw,'ETA','lonAC','latAC',nteta,TIMEeta,'ETAwithSVB.nc')

In [ ]:
maskcw=dsw.maskInC==False

In [ ]:
depth = dsw.Depth
LAT = dsw.YC
LON = dsw.XC-360

In [ ]:
etaw=dsw.ETAN
etan=dsn.ETAN


Zeta = dsw.Z
TIMEeta=dsw.time
nteta = np.size(TIMEeta)

In [ ]:
iters = np.arange(2880,14400,20)
dsn=open_mdsdataset(data_dirNO,data_dirNO,prefix=['eta'],default_dtype='>f4',iters=iters)

In [ ]:
iters = np.arange(2880,14400,20)

dsw=open_mdsdataset(data_dirWITH,data_dirWITH,prefix=['eta'],default_dtype='>f4',iters=iters)

## Plot the points

In [ ]:
varname='SALT'
i=1
dsw2 = xr.open_dataset('/Volumes/Trillian/SVB/exp06_512x612x100_ORL/01_noSVB_febTS/'+ str(varname)+'ACnoSVB'+ str(2+i)+'_'+ str(3+i) +'.nc')

In [ ]:
hfa = np.ma.masked_values(hFacCusew, 71)
maskw = np.ma.getmask(hfa)

In [ ]:
depth = dsw.Depth
index=dsw2.latAC[32*9]
lon_fix=dsw2.lonAC
lat_fix=dsw2.latAC

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,9))
ax.set_facecolor('tan')
pc = ax.contourf(LON,LAT,np.ma.masked_array(depth, mask=maskw),50,
                 vmin=0, vmax=5000, cmap=cmocean.cm.deep)#, extend='max')
cb = plt.colorbar(pc, extend='max',label='depth / m')
cn = ax.contour(LON,LAT,depth, colors=['0.3','0.6'], 
                levels=[250,500])
ax.axhline(y=LAT[index],color='k')
#ax.axhline(y=31.4,color='k')
#ax.axhline(y=30.7,color='k')
#ax.axhline(y=30,color='k')

for ii,jj in zip(lon_fix[2:],lat_fix[2:]):

    ax.plot(LON[ii-1],LAT[jj-1],'o', 
            markersize=4, color='r')
    #print('Depth at cell is %1.2f m' % (depth[jj-1,ii-1]))

#for ii,jj in zip(lon_inds_2,lat_inds_2):
#    ax.plot(LON[ii],LAT[jj-1],'o', 
#            markersize=4, color='y')

ax.plot(LON[lon_fix[0]],LAT[lat_fix[0]],'o', 
        markersize=4, color='b')


cb.set_label('Depth (m)')
ax.set_xlabel('Lon [°]')
ax.set_ylabel('Lat [°]')
ax.set_xlim(238-360, 246-360)
ax.set_ylim(27,35.3)
ax.set_aspect(1)

In [ ]:
lat_fix, lon_fix=findlonlat(hFacCuse,71)

In [ ]:
dscoastn = xr.open_dataset("ETAnoSVB.nc")

In [ ]:
plt.pcolormesh(dat.date2num(dscoastn.time),dscoastn.x,dscoastn.ETA.transpose())

In [ ]:
np.len()